In [1]:
import pandas as pd
import numpy as np
import time
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
df=pd.read_csv('/Users/abhineethmishra/Downloads/train_5UKooLv.csv')

In [2]:
df.head(10)

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,27270,27270,7,01/12/10 8:26,2.55,PX,85123AY
1,27270,27270,7,01/12/10 8:26,3.39,PX,71053R
2,27270,27270,9,01/12/10 8:26,2.75,PX,84406BH
3,27270,27270,7,01/12/10 8:26,3.39,PX,84029GV
4,27270,27270,7,01/12/10 8:26,3.39,PX,84029EX
5,27270,27270,2,01/12/10 8:26,7.65,PX,22752G
6,27270,27270,7,01/12/10 8:26,4.25,PX,21730R
7,427266,427266,3,01/12/10 8:35,5.95,PX,21756Q
8,462735,462735,28,01/12/10 8:45,3.75,RA,22728B
9,462735,462735,28,01/12/10 8:45,3.75,RA,22727B


In [3]:
#removing unnecassary columns
df=df.drop(columns=['InvoiceDate','InvoiceNo','UnitPrice','Country'])

In [4]:
newdf=df
newdf.head()

,CustomerID,Quantity,StockCode
0,27270,7,85123AY
1,27270,7,71053R
2,27270,9,84406BH
3,27270,7,84029GV
4,27270,7,84029EX


In [5]:
#algo = SVD()
data=newdf
#cross_validate(algo,data,measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [6]:
!pip install virtualenv

In [7]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (19.0.3)


In [8]:
!cd ~
!virtualenv venv

# Active your virtual environment
!source pythonenv/bin/activate

Using base prefix '/Library/Frameworks/Python.framework/Versions/3.7'
New python executable in /Users/abhineethmishra/Downloads/venv/bin/python3.7
Also creating executable in /Users/abhineethmishra/Downloads/venv/bin/python
Installing setuptools, pip, wheel...
done.
/bin/sh: pythonenv/bin/activate: No such file or directory


In [11]:
!```shell` pip3 install -U turicreate

/bin/sh: shell: command not found
  Using cached https://files.pythonhosted.org/packages/db/54/167837569bcb816b3fe68f003f18d07ab9d5ac31b2b12b8f9b07b1ccc7a4/turicreate-4.2.tar.gz
  Running setup.py install for turicreate ... error
    Complete output from command /Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -u -c "import setuptools, tokenize;__file__='/private/var/folders/21/x2gdvwls52xfg_x7n07mdbpw0000gn/T/pip-install-qffabvtl/turicreate/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/21/x2gdvwls52xfg_x7n07mdbpw0000gn/T/pip-record-8uquu665/install-record.txt --single-version-externally-managed --compile:
    running install
    
    
            ERROR
    
            If you see this message, pip install did not find an available binary package
            for your system. Supported platforms are:
    
            * Linux x86_64 (including 

In [12]:
import turicreate as tc

ModuleNotFoundError: No module named 'turicreate'

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [ ]:
data_dummy.head()

In [ ]:
#Normalizing using user-item matrix
df_matrix = pd.pivot_table(data, values='Quantity', index='CustomerID', columns='StockCode')
#normalizing
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [ ]:
# create a table for input to the modeling  
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

In [ ]:
df_matrix_norm.shape

In [ ]:
#putiing this into a function
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='Quantity', index='CustomerID', columns='StockCode')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()

In [ ]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = 0.2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [ ]:
customers=[]
customers=newdf['CustomerID'].unique()

In [ ]:
#Defining the model using turicreate
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = customers
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [ ]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
name = 'popularity'
target = 'Quantity'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'cosine'
target = 'Quantity'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'pearson'
target = 'Quantity'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [ ]:
'''Model Evaluation'''
#initial variables
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)